In [32]:
import pandas as pd
import numpy as np 
from scipy.optimize import minimize
import matplotlib.pyplot as plt 

expected_returns = pd.read_csv("../Reports/expected_returns.csv",index_col=0)
cov_matrix = pd.read_csv("../Reports/cov_matrix.csv",index_col=0)
optimal_weights = pd.read_csv("../Reports/Sector_constrained_portfolio.csv",index_col=0)
tickers = expected_returns.index
w_old = pd.read_csv("../Reports/Sector_constrained_portfolio.csv",index_col = 0)
w_old = w_old['Max Sharpe Weights']
# w_old = np.zeros(len(expected_returns))

In [33]:
import yfinance as yf 
from collections import defaultdict

sector_mapping = {
    ticker : yf.Ticker(ticker).info.get('sector') for ticker in tickers
}


sector_indices = defaultdict(list)

# index-based grouping
for i, stock in enumerate(sector_mapping):
    sector = sector_mapping[stock]
    sector_indices[sector].append(i)
custom_sector_constraints = {
    'Technology': {'min': 0.05, 'max': 0.40},
    'Financial Services': {'min': 0.05, 'max': 0.40},
    'Energy': {'min': 0.05, 'max': 0.40}
}

sector_constraints = []

for sector, indices in sector_indices.items():
    if sector in custom_sector_constraints:
        min_limit = custom_sector_constraints[sector]['min']
        max_limit = custom_sector_constraints[sector]['max']

        sector_constraints.append({
            'type': 'ineq',
            'fun': lambda w, idx=indices, max_val=max_limit: max_val - np.sum(w[idx])
        })

        sector_constraints.append({
            'type': 'ineq',
            'fun': lambda w, idx=indices, min_val=min_limit: np.sum(w[idx]) - min_val
        })



all_constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
] + sector_constraints

base_constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
]
base_constraints

[{'type': 'eq', 'fun': <function __main__.<lambda>(w)>}]

In [34]:
strategies = {
    "base": {
        "constraints": base_constraints,
        "use_transaction_cost": False,
    },
    "sector_only": {
        "constraints": all_constraints,
        "use_transaction_cost": False,
    },
    "tc_only": {
        "constraints": base_constraints,
        "use_transaction_cost": True,
    },
    "combined": {
        "constraints": all_constraints,
        "use_transaction_cost": True,
    }
}


In [35]:
results = {}
risk_free_rate = 0.06
alpha = 10

transaction_costs = np.ones(len(expected_returns)) * 0.005 
def portfolio_return(weights, expected_returns):
    return np.dot(weights, expected_returns)

def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

expected_returns_1d = expected_returns.values.flatten()

for name, config in strategies.items():
    print(f"Running strategy: {name}")

    def objective(weights,risk_free_rate):
        ret = portfolio_return(weights,expected_returns)
        vol = portfolio_volatility(weights,cov_matrix)
        sharpe = (ret - risk_free_rate)/ vol
        
        penalty = 0
        if config["use_transaction_cost"]:
            penalty = alpha * np.sum(transaction_costs * np.abs(weights - w_old))

        return -sharpe + penalty  

    res = minimize(
        objective,
        x0=np.ones(len(expected_returns_1d)) / len(expected_returns_1d),
        bounds=[(0.05, 0.3)] * len(expected_returns_1d),
        constraints=config["constraints"],
        method='SLSQP',
        args=(risk_free_rate)
    )

    if res.success:
        w_opt = res.x
        port_ret = np.dot(w_opt, expected_returns_1d)
        port_vol = np.sqrt(np.dot(w_opt.T, np.dot(cov_matrix, w_opt)))
        sharpe = (port_ret - risk_free_rate)/ port_vol
        cost = np.sum(transaction_costs * np.abs(w_opt - w_old)) if config["use_transaction_cost"] else 0

        results[name] = {
            "weights": w_opt,
            "return": port_ret,
            "volatility": port_vol,
            "sharpe": sharpe,
            "transaction_cost": cost
        }
    else:
        print(f"Optimization failed for strategy {name}")
results

Running strategy: base
Running strategy: sector_only
Running strategy: tc_only
Running strategy: combined


{'base': {'weights': array([0.12741636, 0.3       , 0.3       , 0.20679917, 0.06578447]),
  'return': np.float64(0.22661545808751504),
  'volatility': np.float64(0.17100538784330355),
  'sharpe': np.float64(0.9743287050124344),
  'transaction_cost': 0},
 'sector_only': {'weights': array([0.1       , 0.3       , 0.3       , 0.22154062, 0.07845938]),
  'return': np.float64(0.22648320058389723),
  'volatility': np.float64(0.1710017498138176),
  'sharpe': np.float64(0.973576005889764),
  'transaction_cost': 0},
 'tc_only': {'weights': array([0.12741538, 0.3       , 0.3       , 0.20678196, 0.06580267]),
  'return': np.float64(0.22661497342462777),
  'volatility': np.float64(0.17100489215793785),
  'sharpe': np.float64(0.9743286950571238),
  'transaction_cost': np.float64(1.8195622471281482e-07)},
 'combined': {'weights': array([0.1       , 0.3       , 0.3       , 0.22145648, 0.07854352]),
  'return': np.float64(0.22648092469581904),
  'volatility': np.float64(0.17099941059704685),
  'sharpe

In [37]:
import pandas as pd

summary_df = pd.DataFrame.from_dict(results, orient='index')
summary_df = summary_df[['return', 'volatility', 'sharpe', 'transaction_cost']]
summary_df.columns = ['Expected Return', 'Volatility', 'Sharpe Ratio', 'Transaction Cost']
summary_df.to_csv("../Reports/Strategy_comparison.csv")
print(summary_df)

             Expected Return  Volatility  Sharpe Ratio  Transaction Cost
base                0.226615    0.171005      0.974329      0.000000e+00
sector_only         0.226483    0.171002      0.973576      0.000000e+00
tc_only             0.226615    0.171005      0.974329      1.819562e-07
combined            0.226481    0.170999      0.973576      2.741636e-04
